In [ ]:
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

```{autolink-concat}
```

::::{margin}
:::{card} Dispersion integral with rotational cut
TR-980
:::
::::

# Investigation of the dispersion integral with rotation of the branch cut

In [ ]:
%pip install -q numpy==1.24.4 scipy==1.10.1

In [ ]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import quad_vec as integrate

warnings.filterwarnings("ignore")

:::{admonition} General dispersion integral
$$
f(z) = \frac{1}{2\pi i} \int_{-\infty}^\infty \frac{f(z')}{z'- z} \mathop{}\!\mathrm{d}z'
$$
:::

## Dispersion integral with branch cut

In the following a representation of the integral is derived where it should be possible to rotate the branch cut by an arbitrary angle $\phi$ as done in [L. Tiator et al.,Exciting Baryon Resonances on different Riemann Sheets](https://www.google.com/search?client=firefox-b-eq=R.+A.+Arndt%2C+R.+L.+Workman%2C+Z.+Li+et+al.%2C+Phys.+Rev.+C+42%2C+1853+%281990%29). The necessary information regarding the dispersion integral is taken from  [O. Deineka, Dissertation (p.21-22), ](https://openscience.ub.uni-mainz.de/handle/20.500.12030/8998) and [I. Danilkin et al., Data-driven dispersive analysis of the $\pi\pi$ and $KK$ scattering (p.2))](https://journals.aps.org/prd/abstract/10.1103/PhysRevD.103.114023).
## Derivation from line integral 
The $f(z)$ is a complex valued function using Cauchy’s integral formula the line integral around an arbitrary point reads: 
$$
\frac{1}{2\pi i} \oint_{c}\frac{f(z')}{z'- z} \mathop{}\!\mathrm{d}z'
$$

the contour $c$ describes a closed circle around the point. If the function is multivalued the contour
can be modified so that it encircles the functions branch cuts. When neglecting the semi circle part of the contour, (see [O. Deineka, Dissertation (p.21-22), ](https://openscience.ub.uni-mainz.de/handle/20.500.12030/8998) figure 2.6), a this converges to 0 for large $s$, of the contour the integral reads:

$$
f(z) =
\frac{1}{2\pi i}(\int_{z_\mathrm{thr}}^\infty \frac{
    f(z'+i\epsilon)
}{(z'-z)
}\mathop{}\!\mathrm{d}z'+
\int_{\infty}^{z_\mathrm{thr}} \frac{
    f(z'-i\epsilon)
}{(z'-z)
}
\mathop{}\!\mathrm{d}z') =
\frac{1}{2\pi i}(\int_{z_\mathrm{thr}}^\infty \frac{
    (f(z'+i\epsilon) -  f(z'-i\epsilon))
}{(z'-z)
}\mathop{}\!\mathrm{d}z')
$$
To assure the the integral converges a first order subtraction is introduced. This leads to: 
$$
f(z) = f(z_{sp}) +
\frac{(z-z_{sp})}{2\pi i}(\int_{z_\mathrm{thr}}^\infty \frac{
    (f(z'+i\epsilon) -  f(z'-i\epsilon))
}{(z-z_{sp})(z'-z)
}\mathop{}\!\mathrm{d}z')
$$
$z_{sp}$ denotes the subtraction point. 
### Application to the phasespace function
To determine the Chew-Mandelstam function the once subtracted dispersion integral of the phasespace function $\rho(s)$ defined in equation (5) is calculated. The subtraction point is chosen to be the two particle threshold $s_{thr}= (m_1+m_2)²$. 
$$
\rho(s) = \rho(s_\mathrm{thr}) +
\frac{(s-s_\mathrm{thr})}{2\pi i}(\int_{s_\mathrm{thr}}^\infty \frac{
    (\rho(s'+i\epsilon) -  \rho(s'-i\epsilon))
}{(s-s_\mathrm{thr})(s'-s)
}\mathop{}\!\mathrm{d}s')
$$
The function $\rho(s_{thr})$ is chosen to be zero according to [D. J. Wilson et al.,(p.30),Resonances in coupled $\pi K$, $\eta K$ scattering from lattice QCD](https://journals.aps.org/prd/pdf/10.1103/PhysRevD.91.054008).
### Example: Right-hand cut 
A common choice for the direction of the branch cut is from $s_{thr}$ to $+\infty$. The $i\epsilon$ depicts a small complex number above and below the branch cut. As the complex part of the $\rho(s)$ should be 0 below $s_{thr}$ the Schwarz' refection principle applies ([Uni Mainz, Lecture notes, p.13](https://wwwth.kph.uni-mainz.de/files/2019/01/QCD-dispersion-relations.pdf)) as:
$$
\rho(s+ i \epsilon)^*=\rho(s-i \epsilon)
$$
which leads to:
$$
\rho(s+ i \epsilon)-\rho(s-i \epsilon)= \rho(s+ i \epsilon)^* - \rho(s+ i \epsilon)= 2i\rho(s+i\epsilon)
$$
with the assumptions made above the dispersion integral reduces to the formula mentioned in the PDG ([PDG2023, §Resonances, Eq. (50.43)](https://pdg.lbl.gov/2023/reviews/rpp2023-rev-resonances.pdf#page=15)):
$$
\rho(s) =
\frac{(s-s_\mathrm{thr})}{2\pi i}(\int_{s_\mathrm{thr}}^\infty \frac{
    2i\rho(s'+i\epsilon)
}{(s-s_\mathrm{thr})(s'-s)
}\mathop{}\!\mathrm{d}s')=
\frac{(s-s_\mathrm{thr})}{\pi }(\int_{s_\mathrm{thr}}^\infty \frac{
    \rho(s'+i\epsilon)
}{(s-s_\mathrm{thr})(s'-s)
}\mathop{}\!\mathrm{d}s')
$$
### Example: Downwards cut (???)
The direction of the branch cut can be chosen freely. It can for e.g. point in the downwards direction from $s_{thr}$ to $-i\infty$. \epsilon then depicts a small real (???) number left and right from the branch cut. Here the the relation: 
$$
\rho(s+ \epsilon)=-\rho(s-\epsilon)
$$
is used. Which leads:
$$
\rho(s+  \epsilon)-\rho(s-\epsilon)= \rho(s+ \epsilon) - \rho(s+ \epsilon)= 2\rho(s+\epsilon)
$$
$$
\rho(s) =
\frac{(s-s_\mathrm{thr})}{2\pi i}(\int_{-i\infty}^{s_\mathrm{thr}}\frac{
    2\rho(s'+\epsilon)
}{(s-s_\mathrm{thr})(s'-s)
}\mathop{}\!\mathrm{d}s')=
\frac{(s-s_\mathrm{thr})}{\pi i }(\int_{-i\infty}^{s_\mathrm{thr}} \frac{
    \rho(s'+\epsilon)
}{(s-s_\mathrm{thr})(s'-s)
}\mathop{}\!\mathrm{d}s')
$$

### Rotation
The the $s'$ is substituted by:
$$
s' = s_\mathrm{thr} + x e^{i\phi}, \quad
\mathrm{d}s' = e^{i\phi} \mathrm{d}x, \quad
x \in \left[0, \infty\right)
$$
where the $e^{i\phi}$ describes a counter-clockwise rotation in the complex plane by the angle $\phi$. The new integral limits are calculated with:
$$
x(s')=(s'-s_{\mathrm{thr}})e^{-i\phi}
$$
which gives $x(s_{\mathrm{thr}})=0$ and $x(\infty)= \infty  e^{-i\phi}$.
For the right-hand cut follows:
$$
s' = s_\mathrm{thr} + x  \quad
\mathrm{d}s' = \mathrm{d}x
$$
The dispersion integral then reads:
$$
\rho(s) =
\frac{(s-s_\mathrm{thr})}{\pi}(\int_{0}^\infty \frac{
    \rho(s_{thr}+x+i\epsilon)
}{(s-s_\mathrm{thr})((s_{thr}+x)-s)
}\mathop{}\!\mathrm{d}x)
$$
For the downwards cut follows:
$$
s' = s_\mathrm{thr} + xe^{i\frac{3\pi}{2}}= s_\mathrm{thr}+ (-i)x, \quad
\mathrm{d}s' = -i\mathrm{d}x \quad
$$
The dispersion integral then reads:
$$
\rho(s) =
\frac{(s-s_\mathrm{thr})}{\pi i}(\int_{-i\infty }^{0} -i \frac{
    \rho(s_{thr}+(-i)x+\epsilon)
}{(s-s_\mathrm{thr})((s_{thr}+(-i)x)-s)
}\mathop{}\!\mathrm{d}x)
$$

:::{admonition} [PDG2023, §Resonances, Eq. (50.43)](https://pdg.lbl.gov/2023/reviews/rpp2023-rev-resonances.pdf#page=15)

$$
\begin{eqnarray}
\Sigma(s + 0i) &=&
\frac{s-s_\mathrm{thr}}{\pi}
\int_{s_\mathrm{thr}}^\infty \frac{
    \rho(s')
}{
    (s' - s_\mathrm{thr})(s'-s-i0)
}
\mathop{}\!\mathrm{d}s'
\qquad \text{original from the PDG}
\\
&=&
\frac{s-s_\mathrm{thr}}{2\pi}
\left[
\int_{s_\mathrm{thr}}^\infty \frac{
    \rho(s'+0i)
}{
    (s' - s_\mathrm{thr})(s'+i0-s)
}
\mathop{}\!\mathrm{d}s'
-
\int_{s_\mathrm{thr}}^\infty \frac{
    \rho(s'-0i)
}{
    (s' - s_\mathrm{thr})(s'-i0-s)
}
\mathop{}\!\mathrm{d}s'
\right]
\end{eqnarray}
$$

:::

$$
\rho(s) = \frac{2\left|q(s)\right|}{\sqrt[{[\phi]}]{s}}
$$

$$
\sqrt[{[\phi]}]{z} = e^{-\frac{1}{2}\phi i} \sqrt{z e^{\phi i}}
$$

$$
q(s) = \frac{\sqrt{\lambda(s, m_1^2, m_2^2)}}{2\sqrt{s}}
$$

$$
\lambda(x, y, z) = x^2 + y^2 + z^2 - 2xy - 2yz - 2zx
$$

$$
s' = s_\mathrm{thr} - x e^{- i\phi}, \quad
\mathrm{d}s' = -e^{-i\phi} \mathrm{d}x, \quad
x \in \left[0, \infty\right)
$$

$$
\frac{s-s_{\mathrm{thr}}}{2\pi}
\left[
\int^\infty_0 \frac{
    \rho(s_\mathrm{thr} - x e^{-i\phi} + 0i)
}{
    \left(-x e^{-i\phi} + 0i\right)
    \left(s_\mathrm{thr} - x e^{-i\phi} + 0i - s\right)
}
e^{-i\phi}
\mathop{}\!\mathrm{d}x
-
\int^\infty_0 \frac{
    \rho(s_\mathrm{thr} - x e^{-i\phi} - 0i)
}{
    \left(-x e^{-i\phi} - 0i\right)
    \left(s_\mathrm{thr} - x e^{-i\phi} - 0i - s\right)
}
e^{-i\phi}
\mathop{}\!\mathrm{d}x
\right]
$$

In [ ]:
ϵi = 1e-7 * 1j


def rho_cm(s, m1, m2, phi: float = 0) -> np.ndarray:
    return -1j * dispersion_integral(s, m1, m2, phi)


def dispersion_integral(s, m1, m2, phi: float = 0) -> np.ndarray:
    s_thr = (m1 + m2) ** 2
    exp = np.exp(-phi * 1j)
    func1 = (
        lambda x: -rho(x * exp + ϵi, m1, m2, phi)
        / ((-x * exp + ϵi) * (-x * exp + ϵi - s))
        * exp
    )
    func2 = (
        lambda x: -rho(-x * exp - ϵi, m1, m2, phi)
        / ((-x * exp - ϵi) * (-x * exp - ϵi - s))
        * exp
    )
    return (
        (s - s_thr)
        / (2 * np.pi)
        * (integrate(func1, 0, np.inf)[0] - integrate(func2, 0, np.inf)[0])
    )


def rho(s, m1, m2, phi: float = 0) -> np.ndarray:
    return 2 * np.abs(q(s, m1, m2)) / rotated_sqrt(s, phi)


def q(s, m1, m2) -> np.ndarray:
    return np.sqrt(λ(s, m1**2, m2**2)) / (2 * np.sqrt(s))


def λ(x, y, z):
    return x**2 + y**2 + z**2 - 2 * x * y - 2 * y * z - 2 * z * x


def rotated_sqrt(z, phi: float = 0) -> np.ndarray:
    return np.exp(-0.5j * phi) * np.sqrt(z * np.exp(phi * 1j))

In [ ]:
%config InlineBackend.figure_formats = ["svg"]
resolution = 100
x = np.linspace(-2, 2, num=resolution)
m1 = 0.1
m2 = 0.7
phi = np.pi / 6

y1 = rho(x + ϵi, m1, m2, phi)
y2 = rho_cm(x, m1, m2, phi)
assert not any(np.isnan(y2)), "Contains NaN values!"

fig, ax = plt.subplots()
ax.axhline(0, c="black", lw=0.4)
ax.axvline(0, c="black", lw=0.4)
ax.plot(x, y1.real, label=R"$\rho$", lw=4, alpha=0.5)
ax.plot(x, y2.real, label=R"$\rho^\mathrm{CM}$")
ax.axvline((m1 - m2) ** 2, c="red", ls="dotted", label="$(m_1-m_2)^2$")
ax.axvline((m1 + m2) ** 2, c="black", ls="dotted", label="$(m_1+m_2)^2$")
ax.set_ylim(-0.3, +1.6)
ax.set_xlabel("$s$")
ax.set_title(Rf"$\phi={phi / np.pi:.4g}\pi$")
ax.legend()
fig.tight_layout()
plt.show()

In [ ]:
y = np.linspace(-1, +1, num=resolution)
X, Y = np.meshgrid(x, y)
Z = X + Y * 1j
T = rho(Z, m1, m2, phi)

%config InlineBackend.figure_formats = ["png"]
fig, ax = plt.subplots()
ax.set_xlabel(R"$\mathrm{Re}\,s$")
ax.set_ylabel(R"$\mathrm{Im}\,s$")
ax.set_title(Rf"$\phi={phi / np.pi:.4g}\pi$")
T_max = 1.5
mesh = ax.pcolormesh(X, Y, T.real, vmin=-T_max, vmax=+T_max, cmap=plt.cm.coolwarm)
ax.axvline((m1 - m2) ** 2, c="red", ls="dotted", label="$(m_1-m_2)^2$")
ax.axvline((m1 + m2) ** 2, c="black", ls="dotted", label="$(m_1+m_2)^2$")
plt.colorbar(mesh, ax=ax, pad=0.02)
fig.tight_layout()
plt.show()

In [ ]:
y = np.linspace(-1, +1, num=resolution)
X, Y = np.meshgrid(x, y)
Z = X + Y * 1j
T = rho_cm(Z, m1, m2, phi)

%config InlineBackend.figure_formats = ["png"]
fig, ax = plt.subplots()
ax.set_xlabel(R"$\mathrm{Re}\,s$")
ax.set_ylabel(R"$\mathrm{Im}\,s$")
ax.set_title(Rf"$\phi={phi / np.pi:.4g}\pi$")
mesh = ax.pcolormesh(X, Y, T.real, cmap=plt.cm.coolwarm)
plt.colorbar(mesh, ax=ax, pad=0.02)
fig.tight_layout()
plt.show()

:::{error}
Rotating the direction of the integration with $\phi$ **does** rotate the cut in the phase space factor, but also affects the values on the real axis...
:::